#### Import Parent Module 

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters

In [ ]:
import pandas as pd
import os
import ast
import pyflakes

# Constants
QUESTION_TYPE = 1
ANSWER_TYPE = 2

#### LUIGI Parameter

In [ ]:
pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')
output_col = pars.get('output_col')

In [ ]:
dataset = pd.read_csv(input_file, encoding='ISO-8859-1', error_bad_lines=False, sep=";")

#### BlackList

In [ ]:
# answers = answers[:50]
blackList = [22084742, 45695288, 45885112]

def filterBlackList(answers):
    answers = answers[~answers.Id.isin(blackList)]
    return answers

#### Compile Code

TODO: To document...

In [ ]:
def reportCompilability(post, col):
    strcodes = post[col]
    codes = ast.literal_eval(strcodes)
    parseableCount = 0
    total = len(codes)
    errors = []
    errors_type = []
    for code in codes:
        # Compile
        compiled, error, errtype = compileCode(code)
        # Update the report
        errors.append(str(error))
        errors_type.append(errtype)
        if compiled:
            parseableCount += 1
    return parseableCount, total, errors, errors_type


def compileCode(code):
    try:
#         print('Code to compile\n' + code)
        tree = ast.parse(code)
        exec(compile(tree, filename="<ast>", mode="exec"), {})
        return True, "Sucess", "Sucess"
    except Exception as e:
        return False, e, e.__class__.__name__
    
    
def compileQuestion(question, col):
    # Get Answers
    answers = dataset[dataset.ParentId == question.Id]
    answers = filterBlackList(answers)
    
    # Chekc how to log this with LUIGI
    print('----------------------------------------')
    print(question.Id)
    
    count = 0
    total = 0
    question_errors = []
    question_errors_type = []
    for _, answer in answers.iterrows():
        answerCompiled, answerTotal, errors, errors_type = reportCompilability(answer, col)
        print('Checking the answer Id = %s | Errors = %s' % (answer.Id, errors))
        count += answerCompiled
        total += answerTotal
        question_errors.extend(errors)
        question_errors_type.extend(errors_type)
        
    question[output_col + 'Compiled'] = count
    question[output_col + 'Total'] = total
    question[output_col + 'Errors'] = question_errors
    question[output_col + 'Errors Type'] = question_errors_type
    print('----------------------------------------')
    return question

In [ ]:
# Get questions
questions = dataset[dataset.PostTypeId == QUESTION_TYPE]
# Compile
compiledQuestions = questions.apply(compileQuestion, axis=1, args=(input_col,)) 

In [ ]:
def report_errors(analyzedQuestions, prefix):
    errorslist = []
    for err in analyzedQuestions[prefix + 'Errors Type']:
        errorslist.extend(err)
    df = pd.DataFrame(errorslist)   
    return df


#### Save to disk

In [1]:
compiledQuestions.to_csv(output_file, encoding='ISO-8859-1', sep=';', doublequote=True, index=False)